<a href="https://colab.research.google.com/github/fpabi/Tareas/blob/main/Tarea_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install seaborn

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

1 Cargar los datos desde el rchivo csv

In [3]:
csv_path ="/content/datos_ventas.csv"
ventas = pd.read_csv(csv_path)

2 Se calculo el resumen estadistico de ventas

In [4]:
resumen_estadistico = ventas.describe()

In [5]:
print(resumen_estadistico)

         Cantidad      Precio
count  100.000000  100.000000
mean     9.570000   28.888100
std      5.876258   11.189085
min      1.000000   10.660000
25%      5.000000   19.747500
50%      9.000000   27.250000
75%     15.000000   36.842500
max     20.000000   49.790000


3 Analisis por categoria

a)cantidad total de productos vendidos

In [6]:
total_ventas_por_categoria = ventas.pivot_table(index='Categoria', values='Cantidad', aggfunc='sum')
print(total_ventas_por_categoria)

             Cantidad
Categoria            
Categoria 1       462
Categoria 2       495


b) precio promedio de venta

In [7]:
promedio_precios_por_categoria = ventas.groupby("Categoria")["Precio"].mean().reset_index()
print(promedio_precios_por_categoria)

     Categoria     Precio
0  Categoria 1  28.524783
1  Categoria 2  29.197593


c) cantidad maxima y minima de productos vendidos

In [9]:
minimo_cantidad_por_categoria = ventas.groupby("Categoria")["Cantidad"].min().reset_index()
print(minimo_cantidad_por_categoria)

     Categoria  Cantidad
0  Categoria 1         1
1  Categoria 2         1


In [10]:
maximo_cantidad_por_categoria = ventas.groupby("Categoria")["Cantidad"].max().reset_index()
print(maximo_cantidad_por_categoria)

     Categoria  Cantidad
0  Categoria 1        20
1  Categoria 2        20


d) rango intercuartil(IQR)

In [12]:
q_75 = ventas.groupby("Categoria")["Cantidad"].quantile(0.75).reset_index()
print(q_75)

     Categoria  Cantidad
0  Categoria 1      15.0
1  Categoria 2      15.0


In [14]:
q_25 = ventas.groupby("Categoria")["Cantidad"].quantile(0.25).reset_index()
print(q_25)

     Categoria  Cantidad
0  Categoria 1      5.25
1  Categoria 2      4.00


In [23]:
 # Combina los DataFrames q_75 y q_25 usando la columna 'Categoria' como clave
iqr_df = pd.merge(q_75, q_25, on="Categoria", suffixes=("_q75", "_q25"))

# Calcula el IQR restando Q1 de Q3 para cada categoría
iqr_df["IQR"] = iqr_df["Cantidad_q75"] - iqr_df["Cantidad_q25"]

# Imprime el DataFrame resultante
print(iqr_df)

     Categoria  Cantidad_q75  Cantidad_q25    IQR
0  Categoria 1          15.0          5.25   9.75
1  Categoria 2          15.0          4.00  11.00
